<a href="https://colab.research.google.com/github/MWSteams/4590FSP/blob/Residuals/Residuals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Residuals

**Authors: Madeline Keep, Izabella Kornak, Hannah Weinstein**


In [1]:
!pip install aguaclara
!pip install molmass

     |████████████████████████████████| 92kB 3.9MB/s 
     |████████████████████████████████| 163kB 15.8MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.2.1-cp36-none-any.whl size=99466 sha256=1533e9bc20c750f5aff4a89f3aa40b3f4265698140b2b6468f592ef7d17849a5
  Stored in directory: /root/.cache/pip/wheels/7f/ff/4f/10d76e6a261d392aefbdf6ce8d3d5519e924dc334b102ca49b
  Created wheel for pint: filename=Pint-0.8.1-py2.py3-none-any.whl size=134156 sha256=05a7c0c70ece45c29e3be9298b105f4ca24227230b5887341d737cc8b64c1fd4
  Stored in directory: /root/.cache/pip/wheels/b4/56/36/7c4d6a4e3660e233b95e3b0482551915cb184abca0f93ee1dd
Successfully built aguaclara pint
     |████████████████████████████████| 71kB 3.1MB/s 


In [2]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import molmass as mol
!wget https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
import Fairmont as fmt

--2020-05-19 03:13:32--  https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3064 (3.0K) [text/plain]
Saving to: ‘Fairmont.py’

Fairmont.py         100%[===================>]   2.99K  --.-KB/s    in 0s      

2020-05-19 03:13:32 (48.2 MB/s) - ‘Fairmont.py’ saved [3064/3064]



# Task Definition

The overarching goal of Fairmont's residuals management team is to minimize the amount of hazardous waste generated (as hazardous waste is expensive to dispose of and carries a strict cradle-to-grave liability), to limit the energy needed to run our plant, and to decrease the amount of water wasted through the process. We acheieved this by exploring various dewatering processes and treatment options throughout our analysis in order to achieve the dryest cake possible with the lowest energy expenditure.

Our team has decided that the use of rapid mixers and settling tanks are not neccesary in our final design, due to the entirety of our residuals coming from the water treatment plant's own sedimentation tank. Additionally, we have selected a polymer and centrifugation process that will effectively dewater the sludge to a 25% solids per volume estimate. This process helps save plant space, equipment, and energy. The dewatered residuals will then be loaded onto trucks and transported to a hazardous waste disposal site. The leftover water will be recycled to the head of the Fairmont sedimentation plant. 

We will develop a design for the dewatering facility and evaluate its cost and effectiveness. The dewatering facility will be composed of four centrifuges with an average runtime of 18.65 hours per centrifuge per day, which will further dewater the sludge to create a sludge cake. There also must be a distribution mechanism for the sludge cake to be moved out of the facility, which will be achieved through the use of trucks.





# Executive Summary 

As stated in our task definition above, our goal was to develop a complete analysis of the composition, management, and disposal of residuals from the proposed treatment plans. We determined that our design would depend on:
* The composition of the sludge coming from the treatment plant
* Our choice of a thickening and/or dewatering system
* The amount of processed sludge coming from the thickening/dewatering systems

Our constraints include the amount and composition of sludge, the Fairmont Treatment Plant’s design constraints, costs, and California Waste Discharge requirements. Our first step was to determine the sludge composition. By creating a sludge class in Python, we were able to determine the dry sludge mass, wet sludge volume, and wet sludge mass, from the Fairmont design values for flow rate, turbidity reduction, and chemical dosing, and assuming a percent solids by mass of 2%. 
We determined the values as follows:
*  dry sludge mass $= .337 \frac{kg}{s}$
*  wet sludge volume $=16.67 \frac{L}{s}$
*  wet sludge mass $= 16.85 \frac{kg}{s}$

We chose to use Centrysis centrifuges with an added polymer to thicken the sludge. Given the capacity of this centrifuge, we calculated the following information:
* The dewatering capacity can range from 5.363 to 9.464 $\frac{L}{s}$
* We will need between 2 and 4 centrifuges to accomodate our flow
* The sludge will be thickened to a volume between $1.11$ and $2.778\frac{L}{s}$
* This will produce between $1.347 $ and $2.986 \frac{kg}{s}$ of processed residuals
* It will take between 18.65 and 21.14 hours each day to process these residuals

We then determined the amount of loading trucks needed to transport dewatered residuals to disposal sites, as well as the holding capacity to store the residuals in between transportations.

The costs for our design comprise of:
* The hazardous waste disposal costs
* The energy costs of runnning the centrifuges
* The initial purchase cost of the centrifuge
* The centrifuge maintenence costs






# Constraints

##Fairmont Design Constraints
*Constraint information taken from [Fairmont Sedimentation Plant Scope of Work Document](SOWlink) and [MWH](mwhlink)*

* Design flow of the plant has a maximum of 720 cfs and with an average flow of 320 cfs
* Treatment standards are defined in the below table: 

| Contaminant | Design Influent Water Quality  | Fairmont Sedimentation Plant Settled Water Goal | 
|---|---|---|
| Arsenic (μg/L) | 55 | 10 |
| TOC (mg/L)| 4 | 3 |
|Turbidity (NTU) | 23 | 3 |

* Centrifuge capacities
* Unit filter rn volumes (UFRV)
* Unit backwash volume (UBMV)
* Transportation capacities (truck volumes and availability)
* Sludge characteristics
* SOW estimates 144 wet tons of sludge after processing (for off site disposal)


##Costs
* Energy costs
* Transportation costs
* Disposal costs
* Centrifuge costs
* Maintenance costs

##California Waste Discharge Requirements
*Constraint information taken from [The California Water Code](https://www.easylawlookup.com/California-Law/Water-Code/pg-298/_easylookup.blp?data=WATER&sidfw=&site=EASY&location=78232&spon=&stype=P&sterm=+&smode=AND&sexact=ON&pgno=298&par=6901&dlevel=2#par_6901)
and the
[EPA](https://www.epa.gov/sites/production/files/2015-09/documents/train6-residuals.pdf). See appendix for more information.*

* Disposal requirements
  * Hazardous residuals must be disposed of in a hazardous waste landfill
  * Liquid residuals cannot be disposed of in a land disposal unit. To discharge resdiuals directly to a body, the residuals must be non-hazardous and meet National Pollutant Discharge Elimination System (NPDES) discharge requirements.
  * Any mixture or derivative of a hazardous waste is also characterized as a hazardous waste. 


* National TC (toxicity characteristic) for arsenic is 5 mg/L.
* Health-based acceptable loading limits established under 40 CFR 503 are defined in the following table:


Pollutant | Max. Conc (mg/kg) | Monthly Avg Conc (mg/kg) 
--- | --- | --- 
Arsenic| 75 |  41 
Cadmium | 85 | 39 
Copper | 4300 |  1500 
Lead | 840 |  300 
Mercury | 57 | 17 
Molybdenum | 75 |  -- 
Nickel | 420 | 420 
Selenium | 100 | 100 
Zinc | 7500 | 2800 







# Potential strategies to meet dewatering constraints and treatment objectives 


| Strategy | Advantages  | Disadvantages |
|---|---|---|
| Centrifuge | High speed dewatering | High energy input | 
| | 25% and 35% solids by weight| Standby unit needed in some cases |
||Works for all sludge types|Very loud to run|
||Costs less to operate than filter press||
| Plate and Frame Filter Press | Stackable, cake-like solids | High potential for blockage and damage | 
| | 40% and 70% solids by weight | Potential for leakage |
| |Can operate on a continual basis | |
| Belt Filter Press | Effective on high-volume waste streams  | Difficult maintenance |  |
| | 18% and 25% solids by weight | Suitable for high sludge moisture content|
| | Low energy and space taken up| |
||10 year service life||

#Potential strategies to meet sludge handling constraints and objectives

| Strategy | Advantages  | Disadvantages |
|---|---|---|
|Sludge Detention Ponds| Can further thicken sludge | 280,000 sq ft of land required|
| | Lowers volume of sludge in dumping site| High cost of $46000/day| 
|Sludge Processing Facility| Converts hazardous waste into nonhazardous waste | High cost|
| | Has potential to create new materials | Still in research phase, not usable|
|Off-Site Dumping | Lower cost | Hazardous waste dumping | 
| | No extra land or equipment needed | Highest volume of waste produced |


# Shared assumptions

See this [python file, Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py)

Use this file to hold parameters that are in the SOW or elsewhere that
influence the design of the entire plant. This provides an easy mechanism
for all of us to share the same assumptions. Read the SOW to find sections that relate to your part of the design and enter all pertinent variables into [Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py) and then push those changes to github.



# Major Decisions

### Sludge Composition
We can determine the sludge composition based on estimates of the treatment dosages from Fairmont Sedimentation Plant Scope of Work document and the other teams. We calculated the average dose of polymer using the yearly amount of 165 tons/year from the SOW. However, our calculations also require the amount of solids (or percent water composition) as well as the amount of volatile solids in the sludge. We are unable to calculate these exact numbers, and will make assumptions for their values.
### Sludge Dewatering System
Based on our research and calculations, we have to choose which dewatering system will work best in the Fairmont Plant. From the comparisons above, we chose centrifuges as the optimal dewatering technology.
### Polymer addition
A polymer must be added to thickenthe sludge prior to dewatering. We must determine which polymer to use and what dosage, as this choice will affect the sludge composition going into the dewatering centrifuges. We will make this decision based on the type of centrifuge used, size of the centrifuge, the type of influent solids, ionic characteristics of the sludge, water reintroduction potential, and polymer injection point. Based on the Fairmont SOW, we are assuming that the polymer used will be an emulsion polymer.

Based on assumptions detailed in the 
[Polymer Addition section](#Polymer_Addition) below, we will be using a generic emulsion polymer, and have determined the dosage to be .1 milligram polymer per liter of dry sludge




###Centrifuges
The throughput can de adjusted in a centrifuge based on the goal outcomes of percent solids. Our team recommends a range between 80 and 150 gallons per minute to ensure a higher residence time for each unit of sludge which also ensuring the dewatering process is not too timely. 

Aditionally, the differential speed of the decanter needs to be set correctly to ensure the solids are effectively driven through the machine. The beach angle also affects the volume of the centrifuge, which in turn affects the centrifuge's performance. 

### Sludge Handling System
We need to choose a strategy to handle dewatered sludge. We decided to use off-site dumping, as our sludge is hazardous, and because off-site dumping requires the least amount of extra land at the lowest cost. Given that our sludge is hazardous, it must be disposed of in a hazardous waste landfill. The sludge must pass a paint filter test before it can be disposed of in a dry landfill.

### Transportation and Storages
We will have to determine how often we want to remove dewatered residuals from the plant, based on truck capacity and removal cost, and how much storage we will need to accomdate the residuals between removals.










#Physics: Sludge Composition

Our first step is determining the amount and composition of the sludge that is coming from the treatment. We have designed a Sludge class in python to determine the characteristics of the sludge based on the treatment.

Using the total mass of sludge equation, (adapted fromm equation 21-13 in MWH), we can find the weight of the total dry sludge produced from the different parts of treatment:

$$W_s=D_{coagulant}*E_{coagulant}+(\Delta turbidity)*E_{turbidity}+TOC+X$$

$W_s=$ total dry sludge produced,  $kg/m^3$

$\Delta turbidity=$ influent turbidity - goal turbidity

$D_{coagulant}=$ coagulant dose, $kg/m^3$

$E_{coagulant/turbidity}=$ value used to estimate sludge quantities (from table 21-4 in MWH), kg sludge/kg coagulant or turbidity

TOC= total organic carbon $kg/m^3$

X= other coagulant aids

According to the Fairmont SOW, they are using ferric chloride as a coagulant with the addition of a cationic polymer (emulsion polymer), and are also removing turbidity and TOC. 
We are in the process of figuring out how to account for the TOC removed. ***Should the previous sentence be included in final report?***

We then adapted equation Equation 21-3 from MWH to estimate the volume of sludge.

$$V=\frac{W_s}{\rho_{s1}*P_s}$$

$V =$ volume of wet sludge, $m^3$

$W_s=$ weight of total dry solids, kg



$\rho_{sl}=$ density of sludge, $kg/m^3$

$P_s$ = percent solids expressed as a decimal

To determine the specific gravity of sludge, we adapted Equation 21-1:

$$\frac{W_s}{S_s\rho_w}=\frac{W_f}{S_f\rho_w}+\frac{W_v}{S_v\rho_w}$$

$$\frac{W_s}{\rho_s}=\frac{W_f}{\rho_f}+\frac{W_v}{\rho_v}$$

$W_s=$ weight of total dry solids, kg

$\rho_s=$ density of total solids, $kg/m^3$

$W_f=$ weight of fixed solids(mineralmatter),kg

$\rho_f=$ density of fixed solids, $kg/m^3$

$W_v=$ weight of volatile solids, kg

$\rho_v=$ density of volatile solids, $kg/m^3$

With the assumption that the density of the dry volatile solids is approximately 1.0 and the density of the dry fixed solids approximately 2.5.(DEU) The weight of volatile solids will be expressed as a percent of the calculated weight of removed coagulant. We are have used averages based on coagulant type from MWH table 21-5, which is laid out as the chemical/physical properties tables coded below.
###Percent Solids
We currently do not have information on the percent solids composition of our sludge. Without solids input, our code uses the average percent of solids based on the coagulant type from MWH table 21-5. Based on this table, we have assumed that the sludge will be 0.25% solids. This is 4% of the total dry solids per day, so the amount of solids will be expressed as kg/day.
From these equations, we created a sludge class to calculate the mass of dry sludge, and volume and mass of wet sludge.

From this information, we can determine the ideal treatment plan and the design and performance data.

![image.png](https://github.com/MWSteams/4590FSP/raw/master/Images/Residuals/MWHTable21-4.png)

In [3]:
#residual production table 21.2
data_residuals={'Min':[(.08),(4),(2),(1.5),(.08),(.3),(2),(10)],'Max':[.3,8,5,10,.3,6,8,50],'Typical':[.1,'-', ('2 or 3'),(5-8),.1,4,6,(20-30)]}
rows_resid=['Alum','Direct filtration backwash water','Filter backwash water','Ion exchange brine','Iron coagulation sludge','Lime-softening sludge','Microfiltration backwash water','Reverse osmosis concentrate']
data_residproduction=pd.DataFrame(data=data_residuals,index=rows_resid)
#physical properties reference table from table 21-5 asumin max values
#alum, iron
data_physproperties={'Min Volume':[.05,.06],'Max Volume':[.15,.15],'Min Total Solids':[.1,.25], 'Max Total solids':[4,3.5],'Min Dry Density':[1200*u.kg/u.m**3,1200*u.kg/u.m**3],'Max Dry Density': [1500*u.kg/u.m**3,1800*u.kg/u.m**3],'Min Wet Density': [1025*u.kg/u.m**3,1050*u.kg/u.m**3], 'Max Wet Density': [1100*u.kg/u.m**3,1200*u.kg/u.m**3],'Min Spec Resistance':[10110*u.m/u.kg,40440*u.m/u.kg],'Max Spec Resistance':[50550*u.m/u.kg,151650*u.m/u.kg]}
rows=['Alum','Iron']
data_chemconstit={'Min Solids(Al/Fe)':[15,4],'Max Solids(Al/Fe)':[40,21],'Min Solids(Silicates and inert mats)':[35,35],'max Solids(Silicates and inert mats)':[70,70],'Min solids(organics/volatile)':[10,5],'Max solids(organics)':[25,15]}
physical_properties=pd.DataFrame(data=data_physproperties,index=rows)
chemical_constituents=pd.DataFrame(data=data_chemconstit,index=rows)
display(physical_properties)
display(chemical_constituents)

,Min Volume,Max Volume,Min Total Solids,Max Total solids,Min Dry Density,Max Dry Density,Min Wet Density,Max Wet Density,Min Spec Resistance,Max Spec Resistance
Alum,0.05,0.15,0.1,4,1200 kilogram / meter ** 3,1500 kilogram / meter ** 3,1025 kilogram / meter ** 3,1100 kilogram / meter ** 3,1.011e+04 meter / kilogram,5.055e+04 meter / kilogram
Iron,0.06,0.15,0.25,3.5,1200 kilogram / meter ** 3,1800 kilogram / meter ** 3,1050 kilogram / meter ** 3,1200 kilogram / meter ** 3,4.044e+04 meter / kilogram,1.516e+05 meter / kilogram


,Min Solids(Al/Fe),Max Solids(Al/Fe),Min Solids(Silicates and inert mats),max Solids(Silicates and inert mats),Min solids(organics/volatile),Max solids(organics)
Alum,15,40,35,70,10,25
Iron,4,21,35,70,5,15


In [0]:
class sludge:
  def __init__(self,flow):
    self.flow=flow
  def dry_sludge(self,coag_type='None',coag_dose=0*u.mg/u.L,percent_al=0,\
                     influent_turbidity=0*u.NTU,goal_turbidity=0*u.NTU,removal_ratio=1.4*u.g/(u.m**3*u.NTU),\
                     polymer_dose=0*u.mg/u.L):
    if coag_type =='Alum':
      est_val=.26
    elif coag_type =='Ferric sulfate':
      est_val=.53
    elif coag_type =='Ferric chloride':
      est_val=.66
    elif coag_type =='PACl':
      est_val=.0489*percent_al
    else: est_val=0
    self.coag_type=coag_type
    dry_sludge_coag=(self.flow*coag_dose*est_val).to(u.kg/u.day)
    dry_sludge_turbidity=(self.flow*(influent_turbidity-goal_turbidity)*removal_ratio).to(u.kg/u.day)
    dry_sludge_polymer=(self.flow*polymer_dose*1).to(u.kg/u.day)
  #HOW DO WE ACCOUNT FOR TOC
  #influent_TOC=0*u.mg/u.L,goal_TOC=0*u.mg/u.L
    dry_sludge=(dry_sludge_coag+dry_sludge_turbidity+dry_sludge_polymer).to(u.kg/u.day)
    self.dry_sludge=dry_sludge 
    self.coag=dry_sludge_coag
    self.turbidity=dry_sludge_turbidity
    self.polymer=dry_sludge_polymer
    return self.dry_sludge
  def wet_sludge_volume(self,dry_sludge= 0 *u.kg/u.day, temp=15*u.degC,percent_solids=0,rho_fixed=(2.5*1000)*u.kg/(u.m**3),rho_volatile=(1*1000)*u.kg/(u.m**3),volatile_solids=0*u.kg/u.day):
    #REPLACE THIS HERE WHEN WE FIND OUT WHAT THE PERCENT SOLIDS
    #REPLACE THIS WITH THE ASSUMPTION THAT THE VOLATILE SOLIDS WEIGHT = DRY WEIGHT OF TOC SLUDGE
    if dry_sludge==0*u.kg/u.day:
      dry_sludge=self.dry_sludge
    else: self.dry_sludge=dry_sludge
    self.percent_solids=percent_solids
    if percent_solids == 0:
      if self.coag_type =='Alum':
        percent_solids=((4+.1)/2)/100
      elif self.coag_type =='Ferric sulfate' or self.coag_type =='Ferric chloride':
        percent_solids=((3.5+.25)/2)/100
      else: percent_solids= .15
    #Add in volatile
    self.volatile=volatile_solids
    if self.volatile==0 *u.kg/u.day:
      if self.coag_type =='Alum':
        self.volatile=(((10+25)/2)/100)*self.coag
      elif self.coag_type =='Ferric sulfate' or self.coag_type =='Ferric chloride':
        self.volatile=(((5+15)/2)/100)*self.coag
      else: self.volatile=.1*self.coag 
    else: self.volatile=volatile_solids
    rho_water=ac.density_water(temp)
    self.rho_water=rho_water
    rho_solids=(self.dry_sludge/(((self.dry_sludge-self.volatile)/(rho_fixed))+(self.volatile/(rho_volatile))))
    rho_sludge_wet=(1/((percent_solids/rho_solids)+(1-percent_solids)/rho_water))
    self.rho_sludge_wet=rho_sludge_wet
    self.rho_solids=rho_solids
    wet_sludge_volume=self.dry_sludge/(self.rho_sludge_wet*percent_solids)
    self.wet_sludge_volume=wet_sludge_volume
    return self.wet_sludge_volume.to(u.m**3/u.day)
  def wet_sludge_mass(self):
    wet_sludge_mass=self.wet_sludge_volume*self.rho_sludge_wet
    self.wet_sludge_mass=wet_sludge_mass
    return self.wet_sludge_mass.to(u.kg/u.day)
  def dry_sludge_volume(self):
    dry_sludge_volume=self.dry_sludge/self.rho_solids
    self.dry_sludge_volume=dry_sludge_volume
    return self.dry_sludge_volume
  def dewatered_volume(self,dewatered_percent):
    dewatered_volume=(self.percent_solids/dewatered_percent)*self.wet_sludge_volume
    rho_dewatered=(1/((dewatered_percent/self.rho_solids)+(1-dewatered_percent)/self.rho_water))
    self.rho_dewatered=rho_dewatered
    dewatered_mass=dewatered_volume*rho_dewatered
    self.dewatered_mass=dewatered_mass
    return dewatered_volume
 
  

#test=sludge(flow=320*u.feet**3/u.s)
#test.dry_sludge(coag_type='Alum',coag_dose=30*u.mg/u.L,influent_turbidity=25*u.NTU,polymer_dose=1*u.mg/u.L)
#print(test.dry_sludge)
#print(test.dry_sludge)
#test.wet_sludge_volume(temp=15*u.degC,percent_solids=.05,rho_fixed=(2.65*1000)*u.kg/(u.m**3),rho_volatile=(1.05*1000)*u.kg/(u.m**3),volatile_solids=.06*test.dry_sludge)
#print(test.rho_sludge_wet)
#print(test.wet_sludge_volume)
#print(test.coag)
#(test.dry_sludge/(320*u.ft**3/u.sec)).to_base_units()
#volatile_solids=.06*test.drysludge)
#test.density_sludge_wet

#Estimated Sludge Composition
Using the following variables, we estimated the sludge composition using the sludge class.
* Flow rate = design average flow rate = 320 cfs
* Influent turbidity = 23 NTU
* Goal Effluent turbidity = 3 NTU
* Ferric chloride dose = 13.14 mg/L 
* Polymer dose = 0.5235 mg/L
* Percent solids = 2 % (based on table 21-5)

According to the SOW, the dewatering and thickening process should produce 144 wet tons of residuals. From this number, we calculated the percent of water that must be removed during this process.


In [5]:
#our estimate
percent_solids=.02
sludge_estimate=sludge(flow=fmt.q_ave)
dry_sludge=sludge_estimate.dry_sludge(coag_type='Ferric chloride',coag_dose=fmt.Fe_c_ave,influent_turbidity=fmt.turbidity_c_in_design,goal_turbidity=fmt.turbidity_c_out_design,polymer_dose=fmt.emulsionpolymer_c_ave)
wet_sludge_volume=sludge_estimate.wet_sludge_volume(percent_solids=percent_solids)
wet_sludge_mass=sludge_estimate.wet_sludge_mass()
dry_sludge_volume=sludge_estimate.dry_sludge_volume()
print('We estimate that the dry sludge mass produced will be', dry_sludge.to(u.US_ton/u.second),'or',dry_sludge.to(u.kg/u.second))

print('We estimate that the wet sludge volume will be', wet_sludge_volume.to(u.gallon/u.second),'or', wet_sludge_volume.to(u.L/u.second))
print('We estimate that the wet sludge mass will be', wet_sludge_mass.to(u.ton/u.second),'or', wet_sludge_mass.to(u.kg/u.second))
print('We will have to handle', wet_sludge_mass.to(u.ton/u.day),'of wet sludge')
print('The sludge will be', ((wet_sludge_volume/fmt.q_ave).to_base_units())*100, 'percent of the flow of the plant')



We estimate that the dry sludge mass produced will be 0.0003715 US_ton / second or 0.337 kilogram / second
We estimate that the wet sludge volume will be 4.404 gallon / second or 16.67 liter / second
We estimate that the wet sludge mass will be 0.01858 US_ton / second or 16.85 kilogram / second
We will have to handle 1605 US_ton / day of wet sludge
The sludge will be 0.184 dimensionless percent of the flow of the plant


#Preliminary Schematic and Process Descriptions

The imagine below represents our resdiduals treatment and disposal
 schematic 
![image.png](https://github.com/MWSteams/4590FSP/raw/master/Images/Residuals/treatment_train_diagram.png)

1.   Equalization Basin: Provides consistent influent flow to downstream processes by retaining high flow fluctuations.
  *   Requires aeration and mixing (due to the additional retention time)
2. Holding Tank: Holds condensed sludge to maintain centrifugation loading rates.
3. Decanter Centrifuge: Further thickens solids into a cake.
  *  Need to use a decanter centrifuge to decrease the water content of the sludge. 
  *  Treatment rate: 85-150 gpm
  *  There is continuous recovery of the clarified liquid (centrate) by overflow, and continuous gravity discharge of dewatered solids.

4.  Throughout the process, waster is reintroduced to the head of the plant.
  * This dewatered sludge is not expected to carry heavy contaiminants, as the main purpose of this plant is to treat arsenic laden waters through the application of ferric chloride.  

5. Off site disposal: Solid residual cakes, with a solids concentration of 20-30%, will be be hauled off in trucks to be disposed of off site.

6. Backup Generator: Used to ensure that the plant has enough power to run during a power outtage. Generally, power goals will meet a week of operation.

<a id='Polymer_Addition'></a>
#Polymer Addition

A polymer should be added for thickening, which will vary on a number of different factors. This includes the type of centrifuge used, size of the centrifuge, the type of influent solids, ionic characteristics of the sludge, water reintroduction potential, and polymer injection point. The dosing should always be based upon lbs/ton of solids coming in, which may fluctuate. We estimate a fluctuation of about one percent, and for this report have assumed a value of 1.5% to 2.5% percent solids.

The centrisys centrifuge that we picked for this project handles a lot of energy, which can cause high turbulence and speed in the machine. This requires a robust, high strength, cross length polymer that will not detach from the solids mid-process. 

However, it is important to keep in mind that a robust polymer tends to only accept a short window of input solids concentration fluctuations. Many problems can arise if a polymer is only tested at the perfect solids concentration. If a small variance occurs in the real world, the whole process can fall apart quickly. Therefore, solids concentration should be accurately measured and tested thoroughly to ensure that the best polymer is being used. Information regarding polymer reactions to fluctuations in solids concentration can be found online and tested in the lab. An additional consideration includes observing the impact that overdosing a polymer has on coagulation potential of the mix, which can help determine if that polymer would be beneficial for use on highly fluctuating inflows. 

Ferric sludge, which is what is being dealt with at the Fairmont plant, is easy to dewater. This lets the plant get away with using a cheaper polymer as compared to traditional blends. Overall, it is important to understand the balance of requiring more polymer for cheaper varieties vs less polymer for more expensive varieties to yield similar results. City projects tend to rely on cheaper polymers, however, it is very important to do lab testing and realize the effects of different polymers. Testing involves spinning samples in a centrifuge with different polymer types and solids concentrations. We recommend using the centrysis company to do testing because they can use their centrifuge to directly test Fairmont sludge. Additionally, centrysis would not actually be selling the polymer product, which ensures an unbiased suggestion, and the best possible option. Contact information for a California based worker managing these operations is listed in the Vendor Information.

The volume and beach angle of the centrifuge also impacts polymer addition requirements. Every increase in volume is multiplied by an increase in the G-force of the centrifuge, creating something known as a "G Volume". A higher G Volume allows for improved floc settling, which results in a lower polymer requirement. Beach angle, or steepness, also affects volume of the centrifuge. A steeper angle allows for a higher volume, and in turn, a higher settling rate. Keeping these in mind, it is important to note changes in centrifuge design specifications, and how polymer dosing should be altered to account for these changes. 

As usual, it is important to understand the ionic properties of the sludge, as the point of the polymer is to attract flocs. Always choose anionic polymers with cationic flocs, and vice versa. Another consideration includes the form of the polymer. Very large facilities tend to use dry polymer as it comes in pure solid form and requires less storage volume and transportation costs. Emulsion polymers, on the other hand, are already absorbed into an oil or liquid, which means that they are already unraveled and ready to apply. Dry polymers need a makedown system in order to properly unravel. If this step is not considered, the polymer may not be as effective as advertised, and there should be considerations toward loss in the whole system. 

This brings the conversation towards considering injection points into the system. Some option inlude right before centrifuging, directly into the feed pump, in the holding tank, or even in the equalization basins. This is all determined by the type of polymer used. Fragile polymers should be fed into the solution right before the centrifuge. Robust or dry polymers need more retention time, or time to mix, and so they must be inserted earlier in the process.

Water treatment plants that deal with the reintriduction of dewatering liquid into the head of the plant will likely require an National Science Foundation (NSF) approved polymer, which drastically limits the possibilities of polymers in the plant. Since this is one of our team's main goals in the plant, this will be an important consideration.

Based on our limited knowledge, we can only recommend a generic polymer and dosage for the plant. Please note that these calculations are intended to provide insight on approximate costs and quanitity amounts, but should not be directly followed without prior testing. When nonionic or anionic polymers are used as coagulant aids with inorganic coagulants, dosages in the range of 0.1 to 0.5 parts per million are common, while dosages from 0.1 to 0.2 parts per million are usually sufficient for most waters, according to [Tramfloc](http://tramfloc.com/polymers-water-clarification/). Since the coagulant being used is ferric chloride, our team expects a lower dosage to go a long way in the residuals process. Additionally, anionic and nonionic polymers are generally less toxic than cationic polymers, which can block fish gills and cause suffocation. Therefore, minimum dosage to meet flocullation goals should be understood and applied. Based on the SOW, we are assuming that the polymer used will be an emulsion polymer.

Our initial plan was to use the emulsion polymer dosage from the SOW:

In [6]:
Polymer_dosage_estimate=fmt.yearly_emulsion_polymer_est
dose=Polymer_dosage_estimate/dry_sludge
print('Using the SOW emulsion polymer dose, the polymer dosage per dry solids would be ', dose.to(u.g/u.kg))

Using the SOW emulsion polymer dose, the polymer dosage per dry solids would be  14.07 gram / kilogram


However, this dosage is clearly way too high. So, we shifted towards using concentrating on the common values listed in the Tramfloc website as guidance. We settled with a polymer dosage at .1 ppm, or .1 mg/kg, based on our plant and sludge characteristics. We went towards a lower-middle range due to the centrysis centrifuge dewatering capabilities, and the coagulant type used in floculation and sedimentation being ferric chloride. Again, this is an approximate value and should not be applied before proper testing is completed. 

In [0]:
Polymer_dosage= .1*u.mg/u.kg


#Centrifuges

Information and images from [EPA](https://nepis.epa.gov/Exe/ZyNET.exe/2000CAKU.txt?ZyActionD=ZyDocument&Client=EPA&Index=1986%20Thru%201990&Docs=&Query=&Time=&EndTime=&SearchMethod=1&TocRestrict=n&Toc=&TocEntry=&QField=&QFieldYear=&QFieldMonth=&QFieldDay=&UseQField=&IntQFieldOp=0&ExtQFieldOp=0&XmlQuery=&File=D%3A%5CZYFILES%5CINDEX%20DATA%5C86THRU90%5CTXT%5C00000000%5C2000CAKU.txt&User=ANONYMOUS&Password=anonymous&SortMethod=h%7C-&MaximumDocuments=1&FuzzyDegree=0&ImageQuality=r75g8/r75g8/x150y150g16/i425&Display=hpfr&DefSeekPage=x&SearchBack=ZyActionL&Back=ZyActionS&BackDesc=Results%20page&MaximumPages=1&ZyEntry=1)

Solid bowl centrifuges designed for large-scale municipal use are generally 25 feet long and 14 feet wide. These dimensions account for the rotating assembly, backdrive, and adjacent motor. Centrifuges used to dewater sludge can accomodate flow rates of 100-200 gallons per minute (gpm), while centrifuges used to thicken sludge can accomodate flow rates of 400-600 gpm. 

Solid bowl centrifuges come in two designs, a concurrent centrifuge and a countercurrent centrifuge. 

| Concurrent Centrifuge | Countercurrent Centrifuge  |
|---|---|
|Sludge feed introduced into rotating assembly though cylindrical end of bowl |Sludge feed introduced through conical end of bowl |
|Helix orientation draws settle solids in direction of sludge feed towards conical portion of centrifuge|Helix orientation draws solids against direction of sludge feed toward sludge discharge| 


a) Solid bowl concurrent centrifuge


![alt text](https://github.com/MWSteams/4590FSP/raw/master/Images/Residuals/%20Solid_bowl_concurrent_centrifuge.png)

b) Solid bowl countercurrent centrifuge 


![alt text](https://github.com/MWSteams/4590FSP/raw/master/Images/Residuals/Solid_bowl_countercurrent_centrifuge.png)

Solid bowl centrifuges can be either low-speed (operate at gravitational forces less than 1100 g) or high-speed (operate at gravitational forces greater than 1100 g). 

| Strategy | Advantages  | Disadvantages |
|---|---|---|
| Low-speed centrifuge | Lower energy consumption | Lower performance | 
| | Less noisy | Lower capacity to dewater sludge |
||Lower maintenance costs|Needs polymer conditioning|
| High-speed centrifuge | Increased performance | Higher energy consumption | 
| | Increased capacity to dewater sludge (speeds up process) | Higher maintenance costs |
| |Reduces need for polymer conditioning | |

Although high-speed centrifuges have a higher maintence cost, they are now being constructed from improved wear-resistant materials which has lowered the amount of maintenance these centrifuges require. As such, high-speed centrifuges have gained popularity in the realm of municipal residual dewatering.  

The primary centrifuge we are recommending, the Centrisys brand, is a high-speed, concurrent centrifuge. 


#Available Centrifuges

##Thickening
###[Centrisys Sludge Thickener THK350](https://cdn2.hubspot.net/hubfs/4299619/6-1%20Literature/EQUIPMENT%20THK%20Series%20Centrisys.pdf)
* Does not require polymer addition for thickening
* Maximum feed flow rate: 320 gpm
* Average feed flow rate: 265 gpm
* Performance:

| Sludge Thickener | Performance | Waste Activated Sludge  |
|---|---|---|
| No polymer (kW/gpm)|  Min | .07 |
| - |  Max | .18 |
| -|  Avg | .12 |
| With polymer (kW/gpm)|  Min | .05 |
| -|  Min | .15 |
| -|  Min | .08 |
|Avg Solids Recovery (% wt/wt)| w polymer| 99
|-| w/out polymer| 93
|Avg Cake Solids  (% total solids)| w polymer| 5-10
|-| w/out polymer| 4-7

###[Flottweg X5E-4](https://www.flottweg.com/fileadmin/user_upload/data/pdf-downloads/Klaerschlamm-EN.pdf)
* Flow capacity: 130-300 gpm
* Dimensions: 162“ x 61“ x 48“
* Motor for bowl drive: 45 – 75 kW 
* Motor for scroll drive: 15 kW

## Dewatering
### [Centrisys Dewatering Centrifuge CS26-4](https://cdn2.hubspot.net/hubfs/4299619/6-1%20Literature/EQUIPMENT%20CS%20Series%20Centrisys.pdf)

* Flow capacity: 200 to 400 gpm
* 225'' x 56'' x 62''
* Up to 40% cake dryness


###[Flottweg C5E-4](https://www.flottweg.com/fileadmin/user_upload/data/pdf-downloads/Klaerschlamm-EN.pdf)
* Flow capacity: 200-400 gpm
* Dimensions: 161“ x 60“ x 48“
* Motor for bowl drive: 45 – 75 kW 
* Motor for scroll drive: 15 kW

###[Hiller DecaPress DP66](http://www.hiller-us.com/decapress.php)
* Flow capacity: up to 300 gpm
* Bowl ratio: 4.2:1 L/D
* dimensions: 26” x 109”
* Max G: 3,100 “G”




#Centrifuge Considerations
##Design Considerations
*Information from [SNF](https://www.snf.us/wp-content/uploads/2014/08/Sludge-Dewatering-E.pdf) and [Hiller US](http://www.hiller-us.com/sludge-dewatering.php)*
* Plant flow rate
* Sludge produced per day
* Seasonal variations
* Volume of thickened solids to be dewatered
* Range of solids concentration
* Operation hours and times
* Capacity to handle future increases and changes in sewer discharges


The percent solids removal depends on:
1. Feed rate
2. Surface area to depth ratio
3. Design efficiency
4. Feed solids concentration
5. Solids characteristics
6. Coagulant dose

More information on the dry solids estimates can be found in the appendix.

##Centrifuge Capacities
Below, we have calculated the daily maximum and minimum dewaterng capacities of Fairmont WTP's centrifuges.

To calculate the number of centrifuges needed to dewater Fairmont's residuals, we divide the maximum daily flow rate of the Fairmont WTP by the the maximum daily flow rate one centrifuge can handle.

According to our research, dewatering centrifuges will dewater the sludge anywhere from 12 to 30% ([Ontario Design](https://www.ontario.ca/document/design-guidelines-sewage-works/sludge-thickening-and-dewatering)). Based on this value, we estimated the amount of processed sludge we will have.



In [15]:
#Centrifuge Dewatering capacity(85-150 gpm)
dewatering_capacity_min=85*(u.gallon/u.minute)
dewatering_capacity_max=150*(u.gallon/u.minute) 
print('An estimate for the amount of residuals that a centrifuge can dewater is', dewatering_capacity_min.to(u.gallon/u.day), 'to', dewatering_capacity_max.to(u.gallon/u.day),'or', dewatering_capacity_min.to(u.L/u.s), 'to', dewatering_capacity_max.to(u.L/u.s))

#Number of centrifuges for dewatering
min_centrifuges_dewatering=(wet_sludge_volume/dewatering_capacity_max).to_base_units()
max_centrifuges_dewatering=(wet_sludge_volume/dewatering_capacity_min).to_base_units()

print('The number of centrifuges needed to dewater residuals will range from', np.ceil(min(min_centrifuges_dewatering, max_centrifuges_dewatering)),'to',np.ceil(max(min_centrifuges_dewatering, max_centrifuges_dewatering)))
#Using sludge class to determine these values
dewatering_percent_min=.12
dewatering_percent_max=.3
dewatered_sludge_vol_min=sludge_estimate.dewatered_volume(dewatering_percent_max)
dewatered_mass_min=sludge_estimate.dewatered_mass
dewatered_sludge_vol_max=sludge_estimate.dewatered_volume(dewatering_percent_min)
dewatered_mass_max=sludge_estimate.dewatered_mass

print('The sludge will be thickened from a volume of',wet_sludge_volume.to(u.L/u.second),'to a volume between', dewatered_sludge_vol_min.to(u.L/u.second),'and',dewatered_sludge_vol_max.to(u.L/u.second))
print('We will have between',dewatered_mass_min.to(u.US_ton/u.day),'and',dewatered_mass_max.to(u.US_ton/u.day), 'of processed residuals')
print('Or',dewatered_mass_min.to(u.kg/u.second),'and',dewatered_mass_max.to(u.kg/u.second), 'of processed residuals')


An estimate for the amount of residuals that a centrifuge can dewater is 1.224e+05 gallon / day to 2.16e+05 gallon / day or 5.363 liter / second to 9.464 liter / second
The number of centrifuges needed to dewater residuals will range from 2 dimensionless to 4 dimensionless
The sludge will be thickened from a volume of 16.67 liter / second to a volume between 1.111 liter / second and 2.778 liter / second
We will have between 128.3 US_ton / day and 284.4 US_ton / day of processed residuals
Or 1.347 kilogram / second and 2.986 kilogram / second of processed residuals


Given that the Fairmont SOW estimates 144 wet tons of processed sludge per day, this feels like a good estimate to work with.
In order to accomodate the sludge coming through and allow repairs for centrifuges, we will be using 4 centrifuges


# Dewatering Time

Below, we calculate the amount of time each centrifuge must run per day in order to dewater the inputted residuals using the following equation:
$$Time=\frac{V/C}{n_{centrifuges}}$$
$V=$ the wet sludge volume being processed

$C=$ the capacity of a single centrifuge

$n_{centrifuges}=$ the number of centrifuges being utilized



In [9]:
#Amount of time it takes for centrifuges to dewater sludge 
dewatering_time_fast=(wet_sludge_volume/dewatering_capacity_max).to(u.minute/u.day)/2
dewatering_time_slow=(wet_sludge_volume/dewatering_capacity_min).to(u.minute/u.day)/4
avg_dewatering_time=(dewatering_time_fast+dewatering_time_slow)/2
#These values divided by three because 3 centrifgues are required when slowest flow rate is fed into centrifuge
print('The daily time it will take each centrifuge to dewater sludge varies from', dewatering_time_fast.to(u.hour/u.day),'to',dewatering_time_slow.to(u.hour/u.day),'depending on whether two or four centrifuges are utilized') 

The daily time it will take each centrifuge to dewater sludge varies from 21.14 hour / day to 18.65 hour / day depending on whether two or four centrifuges are utilized


#Loading Trucks
Fairmont WTP will utilize 6,500 gallon tankers to transport dewatered residuals
to disposal sites. The code below calculates the number of trucks needed to transport residuals per day based on our estimate of the amount of dewatered residuals produced. We will elect to use the high end of our estimate in order to be prepared for the maximum capacity we might be dealing with.

In [10]:
#must decide if we are doing this calculation in terms of gallons or tons of sludge
truck_capacity=6500*u.gallon
trucks_per_day_max=(dewatered_sludge_vol_max/truck_capacity).to_base_units() 
trucks_per_day_min=(dewatered_sludge_vol_min/truck_capacity).to_base_units() 
print('The number of trucks required to haul dewatered residuals off site will be between',np.ceil(trucks_per_day_min.to(1/u.day)),'and',np.ceil(trucks_per_day_max.to(1/u.day)))


The number of trucks required to haul dewatered residuals off site will be between 4 / day and 10 / day


#Storage Capacity
Given that we only hope to transport the waste every 4 days, we calculated the amount of storage we would need to hold the treated sludge in between transport days.

In [17]:
Holding_capacity=dewatered_sludge_vol_max.to(u.L/u.day)*4*u.day
print('We will need a facility to accomodate', Holding_capacity,'of treated sludge in between transports')

We will need a facility to accomodate 9.602e+05 liter of treated sludge in between transports



# Hazardous Waste Disposal Costs


Information from [Management of Water Treatment Plant Residuals](https://nepis.epa.gov/Exe/ZyNET.exe/30004PI0.TXT?ZyActionD=ZyDocument&Client=EPA&Index=1995+Thru+1999&Docs=&Query=&Time=&EndTime=&SearchMethod=1&TocRestrict=n&Toc=&TocEntry=&QField=&QFieldYear=&QFieldMonth=&QFieldDay=&IntQFieldOp=0&ExtQFieldOp=0&XmlQuery=&File=D%3A%5Czyfiles%5CIndex%20Data%5C95thru99%5CTxt%5C00000003%5C30004PI0.txt&User=ANONYMOUS&Password=anonymous&SortMethod=h%7C-&MaximumDocuments=1&FuzzyDegree=0&ImageQuality=r75g8/r75g8/x150y150g16/i425&Display=hpfr&DefSeekPage=x&SearchBack=ZyActionL&Back=ZyActionS&BackDesc=Results%20page&MaximumPages=1&ZyEntry=1&SeekPage=x&ZyPURL)

The costs associated with hazardous waste disposal include tranportation costs, hazardous waste landfill disposal costs, and stabilization costs. If wastes fail to pass the Paint Filter Liquids Test, they must be stabalized before they are disposed of. 

A PFL test is EPA-approved and is used to determine whether or not there are free liquids in a sample of waste. The use of centrifuges at the Fairmont WTP are to ensure that its residuals will pass the PLF test so that additional costs are not incurred when disposing of the residuals. 

**Hazardous Waste Disposal**

$$Y=[Disposal]+[Transportation]$$

$$Y=[200X]+{[7.9+0.22(Z)][X]}$$


$Y=$ $/year

$X=$ tons of residuals disposed of/year

$Z=$ transportation distance (200-500 miles)


**Stabilization and Hazardous Waste Disposal**

$$Y=[Stabilization/Disposal]+[Transportation]$$

$$Y=[400X]+{[7.9+0.22(Z)][X]}$$


$Y=$ $/year

$X=$ tons of residuals disposed of/year

$Z=$ transportation distance (200-500 miles)
 

In [11]:
#Hazardous waste disposal costs

#min cost
min_tons_residuals_disposed_per_year=dewatered_mass_min.to(u.US_ton/u.day)*365*u.day/u.year*1/u.US_ton #divide by tons so that units are compatible to cost equation
distance=350 #average of 200 and 500 miles
min_hazardous_disposal_cost=(200+7.9+(0.22*distance))*min_tons_residuals_disposed_per_year*u.dollar #multiply by dollar to get correct units


#max cost 
max_tons_residuals_disposed_per_year=dewatered_mass_max.to(u.US_ton/u.day)*365*u.day/u.year*1/u.US_ton #divide by tons so that units are compatible to cost equation
distance=350 #average of 200 and 500 miles
max_hazardous_disposal_cost=(200+7.9+(0.22*distance))*max_tons_residuals_disposed_per_year*u.dollar #multiply by dollar to get correct units

print('The cost of transporting and disposing of hazardous waste residuals is between',min_hazardous_disposal_cost,'and',max_hazardous_disposal_cost,'or',
      min_hazardous_disposal_cost.to(u.dollar/u.day),'and',max_hazardous_disposal_cost.to(u.dollar/u.day))
#costs in the event that our dewatered sludge does not meet PFL test and needs to be stabalized

#min cost
min_tons_residuals_disposed_per_year_stable=dewatered_mass_min.to(u.US_ton/u.day)*365*u.day/u.year*1/u.US_ton #divide by tons so that units are compatible to cost equation
distance=350 #average of 200 and 500 miles
min_hazardous_disposal_cost_stable=(400+7.9+(0.22*distance))*min_tons_residuals_disposed_per_year*u.dollar #multiply by dollar to get correct units


#max cost 
max_tons_residuals_disposed_per_year_stable=dewatered_mass_max.to(u.US_ton/u.day)*365*u.day/u.year*1/u.US_ton #divide by tons so that units are compatible to cost equation
distance=350 #average of 200 and 500 miles
max_hazardous_disposal_cost_stable=(400+7.9+(0.22*distance))*max_tons_residuals_disposed_per_year*u.dollar #multiply by dollar to get correct units


print('The cost of transporting and disposing of hazardous waste residuals that need to be stabilized is between',min_hazardous_disposal_cost_stable,'and',max_hazardous_disposal_cost_stable,'or',
      min_hazardous_disposal_cost_stable.to(u.dollar/u.day),'and',max_hazardous_disposal_cost_stable.to(u.dollar/u.day))


The cost of transporting and disposing of hazardous waste residuals is between 1.334e+07 dollar / year and 2.957e+07 dollar / year or 3.653e+04 dollar / day and 8.096e+04 dollar / day
The cost of transporting and disposing of hazardous waste residuals that need to be stabilized is between 2.271e+07 dollar / year and 5.033e+07 dollar / year or 6.218e+04 dollar / day and 1.378e+05 dollar / day


# Electricity Usage and Costs

Through a conversation with Jared Swanson, the primary contact for the California region of Centrisys Operations, we obtained estimates for lifecycle costs and electricity requirements for each centrifuge of our process.

Electricity consumption varies by centrifuge loading, and can best be calculated by kW/gpm. Optimal loading varies between 0.1 to 0.2 kW/gpm. Highest energy consumption, which comes from very low flow in a typically large centrifuge, can be expected to reach 0.3 kW/gpm. For our purposes, we will calculate energy demands for each scenario in order to be prepared for future requirements. Our baseline throughput will be 150 gpm, the high end of our loading velocity, to ensure that the following calculations do not underestimate eelctricity costs. The caluclations are also based on a centrifuge running time of 18.65 hours per centrifuge per day. These calculations can and should be altered to reflect the changing throughput values within the plant.
Based on these energy requirements, and the assumption that the average cost of electricity is 10 cents/kWh, we calculated the cost of electricity.



In [12]:
Optimal_loading = (0.1*u.kilowatt/(u.gallons/u.minute)).to(u.kilowatt/(u.gallons/u.hour))
Typical_loading = (0.2*u.kilowatt/(u.gallons/u.minute)).to(u.kilowatt/(u.gallons/u.hour))
Poor_loading = (0.3*u.kilowatt/(u.gallons/u.minute)).to(u.kilowatt/(u.gallons/u.hour))

throughput = (150*(u.gallons/u.minute)).to(u.gallons/u.hour)

optimal_energy = Optimal_loading * throughput * dewatering_time_slow.to(u.hour/u.day)*4 #multiply dewatering time by 4 to account for 4 centrifuges
# optimal_cost = cost_per_kWh*optimal_energy
print ('The optimal energy requirement for the four working centrifuges would be', optimal_energy.to(u.kilowatt_hour/u.day))
# print ('The cost of running a centrifuge for eight hours per day at optimal loading is approximated at', optimal_cost )

typical_energy = Typical_loading * throughput * dewatering_time_slow.to(u.hour/u.day)*4 #multiply dewatering time by 4 to account for 4 centrifuges
print ('The typical energy requirement for the four working centrifuges would be', typical_energy.to(u.kilowatt_hour/u.day))

poor_energy = Poor_loading * throughput * dewatering_time_slow.to(u.hour/u.day)*4 #multiply dewatering time by 4 to account for 4 centrifuges
print ('The highest energy requirement for the four working centrifuges would be', poor_energy.to(u.kilowatt_hour/u.day))

#Electricity costs ($/kWh*kWh)

low_electricity_cost=0.1*u.dollar/(u.kilowatt*u.hour)*optimal_energy
print ('The optimal electricity cost to dewater residuals is', low_electricity_cost)

average_electricity_cost=0.1*u.dollar/(u.kilowatt*u.hour)*typical_energy
print ('The typical electricity cost to dewater residuals is', average_electricity_cost)

high_electricity_cost=0.1*u.dollar/(u.kilowatt*u.hour)*poor_energy
print ('The highest electricity cost to dewater resdiuals is', high_electricity_cost)



The optimal energy requirement for the four working centrifuges would be 1119 kilowatt_hour / day
The typical energy requirement for the four working centrifuges would be 2238 kilowatt_hour / day
The highest energy requirement for the four working centrifuges would be 3357 kilowatt_hour / day
The optimal electricity cost to dewater residuals is 111.9 dollar / day
The typical electricity cost to dewater residuals is 223.8 dollar / day
The highest electricity cost to dewater resdiuals is 335.7 dollar / day


# Centrifuge Purchase and Maintenance Costs

An extimated cost we got for each centrifuge is 900,000 dollars. Average operating and maintenance costs for centrifuges vary with hours of use, which we have based on our calculated average value of 15 hours of ontime per day for one centrifuge. Every 10,000-15,000 hours, a minor maintencance issue costing between 10,000 and 25,000 dollara may be expected, and must be accounted for towards costs. Every 20,000-30,000 hours, a "bowl and scroll" fix will cost between 40,000 and 80,000 dollars may be expected. This will not happen every year, but is still beneficial to budget for.

We rounded the above ranges to average values for our calculations, at 18,000 dollars every 12,000 hours, and 60,000 dollars every 25,000 hours of opeeration. These values are based off of centrifuge usage of 18.65 hours per day per centrifuge, given that we will be utilizing four centrifuges in our final disegn, and a given average lifecycle estimate of 20 years. The maintenance costs will be given in an annual sum based off of today's dollars. 

The costs of owning and maintaining all four centrifuges will also be expressed through a twenty year timespan, based on an average inflation rate of 2%. 

In [13]:
#Maintenance Costs

days_minor = (12000/18.65) *u.day  #This value shows us how many days to expect before a minor failure occurs
days_major = (25000/18.65) *u.day  #This value shows us how many days to expect before a minor failure occurs
cost_minor = 18000*u.dollar
cost_major = 60000*u.dollar
days_peryear = 365*u.day/u.year

#Life Cost Analysis

purchase_cost = 900000*u.dollar
lifecycle = 20 *u.years
inflation = 2
num_centrifuges = 4
annual_cost = ((cost_minor * days_peryear /days_minor) + (cost_major * (days_peryear/days_major))).to(u.dollar/u.year)

#annual to present :  P = A  ((1+i)^n - 1)/(i(1+i)^n))
variable = (0.499)/u.year
annual_sums = ((annual_cost/variable) + purchase_cost)*4

print('The cost of buying four centrifuges is', purchase_cost*4)
print('The annual maintenace costs of one centrifuge would be approximated at', annual_cost)
print('The life cycle cost of purchasing and maintaining four centrifuges for twenty years is', annual_sums)

The cost of buying four centrifuges is 3.6e+06 dollar
The annual maintenace costs of one centrifuge would be approximated at 2.655e+04 dollar / year
The life cycle cost of purchasing and maintaining four centrifuges for twenty years is 3.813e+06 dollar


# Vendors and contact information file

See this [tab delimited spreadsheet](https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt).

Add rows to the vendor spreadsheet as you find them. The sheet can easily be edited in Excel. Just make sure to save it as a tab delimited sheet. Push your changes to github so the entire class has access to the document.



In [14]:
vendors = pd.read_csv('https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt',sep='\t')
vendors.head()

,item description,vendor,contact first name,contact last name,vendor website,product info,date of contact,who made the contact,notes
0,plate settlers,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/literaturemedia/...,4/16/2020,Ian Cullings,NaN
1,sludge removal system,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/products/hoseles...,4/16/2020,Ian Cullings,NaN
2,plate settlers,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,https://www.monroeenvironmental.com/water-and-...,4/16/2020,Ian Cullings,NaN
3,sludge removal system,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,http://www.monroeenvironmental.com/water-and-w...,4/16/2020,Ian Cullings,NaN
4,plate settler with hopper,Parkson,still,pending,https://www.parkson.com/,https://www.parkson.com/products/lamella-ecoflow,4/16/2020,Ian Cullings,Includes sloped sludge hopper in design


#Resources

* [EPA Residuals Treatment](https://www.epa.gov/sites/production/files/2015-11/documents/dw-treatment-residuals-mgmt-tech-report-sept-2011.pdf)
* [EPA Draft Methods](https://www.epa.gov/sites/production/files/2015-10/documents/method_1684_draft_2001.pdf)
* [DEU](http://web.deu.edu.tr/atiksu/ana58/c-7.pdf)
* [Vtech](https://vtechworks.lib.vt.edu/bitstream/handle/10919/26604/04Chapter3.pdf?sequence=3)
* [EPA](https://nepis.epa.gov/Exe/ZyNET.exe/30004PI0.txt?ZyActionD=ZyDocument&Client=EPA&Index=1995%20Thru%201999&Docs=&Query=&Time=&EndTime=&SearchMethod=1&TocRestrict=n&Toc=&TocEntry=&QField=&QFieldYear=&QFieldMonth=&QFieldDay=&UseQField=&IntQFieldOp=0&ExtQFieldOp=0&XmlQuery=&File=D%3A%5CZYFILES%5CINDEX%20DATA%5C95THRU99%5CTXT%5C00000003%5C30004PI0.txt&User=ANONYMOUS&Password=anonymous&SortMethod=h%7C-&MaximumDocuments=1&FuzzyDegree=0&ImageQuality=r75g8/r75g8/x150y150g16/i425&Display=hpfr&DefSeekPage=x&SearchBack=ZyActionL&Back=ZyActionS&BackDesc=Results%20page&MaximumPages=1&ZyEntry=213)
* [EPA Residuals Train Methods](https://www.epa.gov/sites/production/files/2015-09/documents/train6-residuals.pdf)
* [EPA Centrifuge Information](https://nepis.epa.gov/Exe/ZyNET.exe/2000CAKU.txt?ZyActionD=ZyDocument&Client=EPA&Index=1986%20Thru%201990&Docs=&Query=&Time=&EndTime=&SearchMethod=1&TocRestrict=n&Toc=&TocEntry=&QField=&QFieldYear=&QFieldMonth=&QFieldDay=&UseQField=&IntQFieldOp=0&ExtQFieldOp=0&XmlQuery=&File=D%3A%5CZYFILES%5CINDEX%20DATA%5C86THRU90%5CTXT%5C00000000%5C2000CAKU.txt&User=ANONYMOUS&Password=anonymous&SortMethod=h%7C-&MaximumDocuments=1&FuzzyDegree=0&ImageQuality=r75g8/r75g8/x150y150g16/i425&Display=hpfr&DefSeekPage=x&SearchBack=ZyActionL&Back=ZyActionS&BackDesc=Results%20page&MaximumPages=1&ZyEntry=3)




#Appendix 
##California Waste Discharge Requirements
*Constraint information taken from [The California Water Code](https://www.easylawlookup.com/California-Law/Water-Code/pg-298/_easylookup.blp?data=WATER&sidfw=&site=EASY&location=78232&spon=&stype=P&sterm=+&smode=AND&sexact=ON&pgno=298&par=6901&dlevel=2#par_6901)
and the
[EPA](https://www.epa.gov/sites/production/files/2015-09/documents/train6-residuals.pdf).*

* Only non hazardous residuals may be disposed in a solid waste landfill, 


The treatment of drinking water results in liquid and solid wastes that contain concentrated contaminants such as arsenic. ([EPA](https://www.epa.gov/sites/production/files/2015-09/documents/train6-residuals.pdf)) The Faimont Water Treatment Plant must dispose of their residuals in a way that complies with California law and minimizes environmental impact. Raw water characteristics that may impact the disposal optins of residuals include:
* Excessively high or low pH;
* High total suspended solids (TSS);
* High total dissolved solids (TDS);
* High concentrations of heavy metals, including arsenic, lead, and aluminum;
* High concentrations of competing ions, including fluoride, sodium, sulfate chloride, and other salts concentrations;
* High concentrations of radionuclides and daughter products.


To dispose of residuals in a solid waste landfill, the residuals must be nonhazardous, otherwise they must be disposed of in a hazardous waste landfill. Liquid residuals cannot be disposed of in a land disposal unit. To discharge resdiuals directly to a body, the residuals must be non-hazardous and meet National Pollutant Discharge Elimination System (NPDES) discharge requirements.


Depending on the concentrations of contanimants in a waste stream, intermediate processing may be neccessary before disposal. Types of **intermediate processing for liquid waste** includes flow equilization (detaining and mixing water system wastes before releasing residuals to a POTW), brine recycling (caution must be exercised, as reused brine streams can possess higher levels of suspended solids and contaminants), pH neutralization for wastes streams with a significantly high or low pH, settling or gravity thickening (removes contaminants from liquid waste stream, resulting in a cleaner liquid waste and a concentrated sludge), evaporation (leaves solids and contaminants behind), and chemical precipitation. Types of **intermediate processing for solid wastes** include mechanical dewatering processes (centrifugal filter presses, vacuumassisted dewatering beds, belt filter presses, plate-and-frame filter presses), filter presses, non-mechanical dewatering processes (lagoons, settling basins, gravity thickening, evaporation ponds, drying beds), and thermal processing (incineration, pyrolysis).


For the most part, the dillution of hazardous waste in not recommended, as any mixture or derivative of a hazardous waste is also characterized as a hazardous waste. 

# Arsenic Standards 

The national TC (toxicity characteristic) for arsenic is 5 mg/L. If the concentration of arsenic in residuals exceeds this value, the waste in considered hazardous and is not acceptable for direct discharge. However, this value can vary region-to-region. The TBLL (technically based local limit) for arsenic is 1 mg/L in San Jose, CA and 2 mg/L in Orange County, CA. 


# Processing of Nonhazardous Residuals


Nonhazardous residuals produced from sorption processes or chemical precipitation may be disposed of in the following ways:

>Recycle backwash water to facility headwokrs, discharge backwash to recieving body or POTW, and dispose of solid wastes in landfill.

Nonhazardous residuals produced from membrane processes may be disposed of in the following ways:

>Solid wastes disposed in landfill, liquid wastes may need pretreatment before discharge into POTW. 



# Health-based acceptable loading limits established under 40 CFR 503:


Pollutant | Ceiling concentration (mg/kg) | Cumulative pollutant loading rate (kg/hectare) | Monthly avg concentration (mg/kg) | Annual Pollutant loading rate (kg/hectare/365 day period)
--- | --- | --- | --- | ---
Arsenic| 75 | 41 | 41 | 2.0 
Cadmium | 85 | 39 | 39 | 1.9 
Copper | 4300 | 1500 | 1500 | 75
Lead | 840 | 300 | 300 | 15
Mercury | 57 | 17 | 17 | 0.85
Molybdenum | 75 | -- | -- | -- 
Nickel | 420 | 420 | 420 | 21
Selenium | 100 | 100 | 100 | 5.0
Zinc | 7500 | 2800 | 2800 | 140



The overall goal of Fairmont's residuals management team is to minimize the amount of hazardous waste generated, as hazardous waste is expensive to dispose of and carries a strict cradle-to-grave liability.

#Dry Solids Estimates and Variables
In addition to inorganic chemicals including metal salts and polyelectrolites, a large variety of organic-based polymers are employed as coagulant aids either independently or in concert with the inorganic coagulation aids. A large number of different formulations of polymer beads, polyacrylamide, polyamines, and polydimethylammonium chloride are listed in ANSI/NSF Standard 60 and can be used to promote the removal of turbidity from drinking water. Some of these chemicals are also referred to as filtration aids, but function the same way as coagulants. Residuals are generated as underflow discharges from the sedimentation tanks. These residuals contain source water contaminants, as well as chemicals added to aid solid removal and formulation impurities in the added treatment chemicals.
Below is a chart formed from data aquired by field testing regarding polymer addition and solids concentration output as it relates to three different polymers.

![image.png](https://github.com/MWSteams/4590FSP/raw/master/Images/Residuals/polymer%20addition%20cake%20dryness.png)

The graph below illustrates how torque can influence the cake dryness of centrifugal solids. Fortunately, most centrifuges have a torque meter that can easily be read, which allows for a direct estimate of the cake dryness values. As the cake dryness increases, it requires more torque to convey or scroll the cake up through the conical section and out the cake discharge ports. 

![image.png](https://github.com/MWSteams/4590FSP/raw/master/Images/Residuals/torque%20cake%20dryness.png)

The ANDRITZ separation company tested out three different polymers, and discovered that Polydyne C 9530 produced the best results, as highlighted in the blue graph.
Furthermore, the highest cake dryness was produced with this polymer, at a dryness of 22%, which will meet our residuals percent solids composition goal of 20-30%. This dryness was acheieved with the addition of 25 active pounds per ton of total solids. This graph also shows us that additional polymer dosing increases cake dryness, but these positive results taper off at a certain dosage. This research can help us determine the amount of polymer that should be added to our process.

The below graph, also provided by research produced by the ANDRITZ company, shows the effects of throughput on solids production. In these terms, throughput represents the amount of sludge passing through the centrifuge. As the throughput increases, the differential speed between the bowl and scroll must be increased to maintain the solids capture rate. This reduction in sludge retention time causes the cake dryness to
decrease as seen in the below graph. Therefore, we can deduce that, though centrifuges can pass 200 gallons per minute, this may not be a flow rate which can help meet our cake dryness goal. As presented below, the percent total solids decreased dramatically between 85 and 115 GPM, and continued to taper downwards. At throughputs between 50 and 85 GPM, there was not a very noticeable difference in effectiveness of dewatering. These results are what prompted us to define our centrifuge feeding rate as 85-150 gpm of residuals, as this range allows for a relatively time-efficient dewatering process while also ensuring optimal dewatering capabilities.  
![image.png](https://github.com/MWSteams/4590FSP/raw/master/Images/Residuals/throughput%20cake%20dryness.png)


#Physics: Centrifuges

$$G=\frac{r\omega^2}{g}$$
$r=$ radius of centrifugal dewatering vessel

$g=$ gravitational acceleration

Using Darcy's law to predict the rate of drainage through the filter cake [1,12]:
$$Q=\frac{K\Delta P A}{\mu L}$$

$Q=$ flow rate

$K=$ permeability of cake

$\Delta P=$ pressure drop across the cake

$A=$ filtration area

$\mu=$ dynamic viscosity of water

$L=$ cake thickness

During filtration period, the pressure drop across the cake is determined by the following relationship:
$$\Delta P=\frac{1}{2}\rho\omega^2(r_s^2-r_0^2)$$

$\rho=$ density of the liquid

$r_0=$ radial distance from the rotation axis of the centerfuge to the free water

$r_s =$ radial distance from the rotation axis of the centerfuge to the cake surface

Centrifugal dewatering is largely used for separating excess water from sludge. The applied centrifugal force created by the angular velocity of the rotating basket allows for w lower moisture content of the product, which is the goal of our team, both to limit the amount of waste that is being dumped, and to best salvage water for reintroduction into the treatment process. 

Centrifugal treatment starts with a process called filtration, in which water flows outwards through the sludge while the sludge is covered in a layer of water. In time, as the water is increasingly removed from the sludge, drainage occurs in the centrifuge. The pressure drop becomes negative during this slower process, as the thickness of the solid cake is increased. 

To start to observe the physical characteristics that outline the process of centrifugal dewatering, we calculated...


From the top equations, we can deduce that the rate of filtration should increase with angular velocity and the thickness of the water
over a filter cake. 

We also see that the change in pressure approaches zero when the water over the cake decreases. As the water level in the cake decreases further, the pressure within the cake becomes lower than the ambient pressure.This may be the fundamental reason that centrifuges cannot produce as low cake moistures as vacuum or pressure filters for
the finer particles.